    사용하고 있는 데이터 세트는 다양한 사람들이 여섯 가지 다른 운동(아래로 내려가기, 조깅, 앉기, 서기, 위로 올라가기, 걷기)을 수행하는 동안 휴대폰에서 수집한 가속도 데이터 모음입니다. 각 운동에 대해 x, y, z 축의 가속도를 측정하여 타임스탬프와 사람 ID와 함께 캡처되었습니다.
    
    이러한 데이터를 사용하여 신경망을 훈련하여 휴대폰을 들고 있는 사람이 여섯 가지 활동 중 하나를 수행하고 있는지 해석합니다. 신경망이 기존 데이터로 훈련되면 이전에 보지 못한 데이터가 주어질 때 사람이 수행하고 있는 활동의 유형을 올바르게 예측할 수 있어야 합니다.
    
    사용 가능한 데이터를 기반으로 DL 모델은 여섯 가지 활동 중 각각을 구별하는 방법을 학습할 것입니다. 그런 다음 신경망에 새로운 데이터를 보여주면 특정 시점에 사용자가 무엇을 하고 있는지 알려줄 것입니다. 이 문제에 대한 해결책은 아래 그림에 나와 있습니다.

<img src ='pics/1.png'>

In [ ]:
#from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.utils import np_utils

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# 일부 파라미터 설정
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
print('keras version ', keras.__version__)

    TIME_PERIODS - length of the time segment
    STEP_DISTANCE - amount of overlap between two consecutive time segments

In [ ]:
# 아래와 같은 라벨을 프로그램 전체에서 사용
LABELS = ['내려가기',
          '달리기',
          '앉기',
          '서기',
          '올라가기',
          '걷기']
# 하나의 time 세그먼트 내에서의 step의 개수
TIME_PERIODS = 80
# 다음 세그먼트로 넘어가기 위한 steps 수. 이 값이 TIME_PERIODS와 동일하면 세그먼트들 사이에 오버랩이 없음
STEP_DISTANCE = 40

### Data 로딩하기

In [ ]:
def read_data(file_path):

    column_names = ['id',
                    '활동',
                    'timestamp',
                    'x-axis',
                    'y-axis',
                    'z-axis']
    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)
    # 마지막 컬럼에 있는 불필요한 ";" 문자 제거
    df['z-axis'].replace(regex=True,
      inplace=True,
      to_replace=r';',
      value=r'')
    # z축은 float으로 명시적 형변환시키기
    df['z-axis'] = df['z-axis'].apply(convert_to_float)
    # loss인 경우 NAN으로 표시되도록
    df.dropna(axis=0, how='any', inplace=True)

    return df

def convert_to_float(x):

    try:
        return np.float(x)
    except:
        return np.nan

def show_basic_dataframe_info(dataframe):
    # 행렬의 개수 출력
    print('dataframe에서 열의 개수: %i' % (dataframe.shape[1]))
    print('dataframe에서 행의 개수:: %i\n' % (dataframe.shape[0]))

In [ ]:
# csv 파일 형태의 data set을 로딩하기
df = read_data('dataset/WISDM_ar_v1.1_raw.txt')

In [ ]:
# data 정보 표시
show_basic_dataframe_info(df)
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

### Exloratory Data Analysis

In [ ]:
# 6개 활동 각각에 대한 train 예제 개수 출력
df['activity'].value_counts().plot(kind='bar',
                                   title='Training Examples by Activity Type')
plt.show()

In [ ]:
# train에 참여한 개체별 레코드 수
df['user-id'].value_counts().plot(kind='bar',
                                  title='Training Examples by ID')
plt.show()

In [ ]:
print(set(df['id']))

In [ ]:
len(set(df['id']))

보시다시피, 다른 활동보다 걷기와 조깅 활동에 대한 데이터가 더 많습니다. 또한 36개체가 실험에 참여한 것을 알 수 있습니다.

### Data 시각화

다음으로 6가지 가능한 활동 모두에 대해 세가지 축 각각에 대한 가속도 데이터를 살펴보겠습니다. 데이터는 20Hz(초당 20값)의 샘플링 속도로 기록됩니다. 처음 180개의 레코드를 표시하므로 각 차트에는 여섯 가지 활동 각각에 대해 9초 간격이 표시됩니다(계산: 0.05 * 180 = 9초).

In [ ]:
def plot_activity(activity, data):

    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3,
         figsize=(15, 10),
         sharex=True)
    plot_axis(ax0, data['timestamp'], data['x-axis'], 'X-Axis')
    plot_axis(ax1, data['timestamp'], data['y-axis'], 'Y-Axis')
    plot_axis(ax2, data['timestamp'], data['z-axis'], 'Z-Axis')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()

def plot_axis(ax, x, y, title):

    ax.plot(x, y, 'r')
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

for activity in np.unique(df['activity']):
    subset = df[df['activity'] == activity][:180]
    plot_activity(activity, subset)

In [ ]:
# label 벡터의 열의 이름을 정의
LABEL = 'ActivityEncoded'
# LabelEncoder를 통해서 String에서 Integer로 labels을 변환
le = preprocessing.LabelEncoder()
# 기존 DataFrame에 대한 인코드 값으로 새로운 열을 추가하기
df[LABEL] = le.fit_transform(df['activity'].values.ravel())

In [ ]:
df.ActivityEncoded.tail(10)

In [ ]:
set(df.ActivityEncoded)

### Train 테스트 분활

잘못된 분할 전략 - 각 활동당 최소 몇 개의 예제 레코드만 있는 것에만 걱정하는 것은 충분하지 않습니다. 훈련 세트에서 사람 5에 대해 "걷기" 활동의 레코드가 3개, 테스트 세트에서 개체 5에 대해 "걷기" 활동의 레코드가 1개 있는 경우와 같은 위험에 처하게 됩니다. 물론 이러한 상황에서는 모델이 훈련 중에 개체 5의 이동 패턴을 이미 보았기 때문에 훌륭하게 수행할 것입니다. 항상 DNN의 성능에 대해 비판적이어야 합니다. 잘못된 데이터 분할 때문에 발생할 수 있습니다.

ID 1부터 28까지의 개체는 모델 훈련에 사용하고 ID가 28보다 큰 개체는 테스트 세트에 사용하겠습니다. 이는 기타 센서 데이터(예: 사물 인터넷)에도 적용됩니다.

In [ ]:
# 테스트 set과 training set 구분하기
df_test = df[df['user-id'] > 28]
df_train = df[df['user-id'] <= 28]

## Training Data 정규화

다음으로, 훈련 데이터 내에서 특징들을 정규화해야 합니다. 물론 정규화하는 방법에는 여러 가지가 있습니다. 나중에 신경망에 새로운 데이터를 입력할 때 동일한 정규화 알고리즘을 사용한다는 점을 기억하십시오. 그렇지 않으면 예측이 벗어날 것입니다. 정규화 외에도 세 가지 특성을 소수점 여섯 자리까지 반올림하기로 했습니다. 네트워크의 성능을 더욱 높이기 위해 여기에서 최적의 수를 가질 수 있습니다.

In [ ]:
# Normalize features for training data set (values between 0 and 1)
max_data = []
for axis in ['x-axis','y-axis','z-axis']:
    max_data.append((axis, df_train[axis].max()))
    df_train[axis] = df_train[axis] / df_train[axis].max()
max_data = dict(max_data)
max_data

In [ ]:
# Round numbers
df_train = df_train.round({'x-axis': 4, 'y-axis': 4, 'z-axis': 4})

## Reshape Data

dataframe내에 포함된 데이터는 아직 신경망에 입력할 준비가 되어 있지 않은 상태입니다. 따라서 데이터를 재구성(reshape)해야 합니다. 이를 위해 "create_segments_and_labels"라는 또 다른 함수를 만들어봅시다. 이 함수는 dataframe과 레이블 이름(처음에 정의한 상수)뿐만 아니라 각 레코드의 길이를 입력으로 받습니다. 여기서는 40 단계로 하겠습니다. 20Hz 샘플링 속도를 고려하면 이는 4초 간격(계산: 0.05 * 80 = 4)에 해당합니다. 함수는 데이터를 재구성하는 것 외에도 특징(x 가속도, y 가속도, z 가속도)과 레이블(관련된 활동)을 분리합니다.

In [ ]:
print(TIME_PERIODS)
print(STEP_DISTANCE)
print(LABEL)

In [ ]:
def create_segments_and_labels(df, time_steps, step, label_name):

    # x, y, z 가속도 이렇게 3개를 features로 사용
    N_FEATURES = 3
    # 각 iteration내에서 steps의 개수. (여기서 time_steps과 동일하다고 가정함. 세그먼트들 사이의 오버랩이 없다고 가정하므로)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['x-axis'].values[i: i + time_steps]
        ys = df['y-axis'].values[i: i + time_steps]
        zs = df['z-axis'].values[i: i + time_steps]
        # 이 세그먼트에서 가장 자주 사용되는 label을 가져오기
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)

    # 세그먼트를 더 나은 형태로 변환
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

In [ ]:
x_train, y_train = create_segments_and_labels(df_train,
                                              TIME_PERIODS,
                                              STEP_DISTANCE,
                                              LABEL)

이제 x_train과 y_train 모두에 20,868개의 레코드가 있어야 합니다. x_train의 20,868개 레코드 각각은 80x3의 크기를 가진 2차원 매트릭스 형태입니다.

In [ ]:
print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)

In [ ]:
#x_train[0]

딥신경망을 구성하기 위해 다음 차원을 저장해야 합니다.:
-  시간 간격 수: 이는 하나의 레코드 내에서의 시간 간격 수입니다 (2초의 시간 간격을 원했기 때문에 이 값은 여기서는 40이 되입니다)
-  센서 수: x, y, z 축의 가속도만 사용하기 때문에 이 값은 3입니다
-  클래스 수: 이는 신경망의 출력 계층에 대한 노드의 수입니다. 신경망이 활동의 유형을 예측하도록 하려면 이전에 사용한 인코더에서 클래스의 수를 가져올 것입니다.


In [ ]:
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
num_classes = le.classes_.size
print(list(le.classes_))

우리가 네트워크에 입력하고 싶은 데이터는 2차원 (80x3) 형태입니다. 불행히도, Keras와 CoreML을 함께 사용하면 다차원 입력 데이터를 처리할 수 없습니다. 따라서, input layer에 입력하기 위해 데이터를 "faltten"하게 만들어야 합니다. 80x3 형태의 행렬을 입력하는 대신, 240개의 값으로 구성된 리스트로 입력합니다.

In [ ]:
input_shape = (num_time_periods*num_sensors)
x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train.shape)
print('input_shape:', input_shape)

계속하기 전에, 모든 특징 데이터(x_train)와 label 데이터(y_train)를 Keras에서 허용하는 데이터형으로 변환해야 합니다.

In [ ]:
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

우리는 데이터 준비를 거의 마쳤습니다. 마지막 단계로 레이블에 one-hot encoding을 수행해야 합니다. 이 줄은 한 번만 실행하세요!

In [ ]:
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print('New y_train shape: ', y_train_hot.shape)

## Keras로 1D CNN Model 생성

이제까지 우리가 해야할 잡다한 일들이 완료되었습니다. 데이터는 Keras가 처리할 수 있는 형식으로 준비되어 있습니다. 각각 100개의 완전 연결된 노드가 있는 3개의 은닉층으로 구성된 신경망을 만들기로 결정했습니다.

중요한 노드: 기억하시겠지만, 입력 데이터를 80x3 행렬에서 길이 240의 벡터로 재구성했습니다. 이를 되돌리기 위해 신경망의 첫 번째 레이어는 데이터를 "이전" 형식으로 재구성합니다. 마지막 두 레이어는 다시 데이터를 flatten 한 다음 softmax 활성화 함수를 실행하여 각 클래스에 대한 확률을 계산합니다. 우리는 우리의 경우에 여섯 개의 클래스(내려가기, 달리기, 앉기, 서기, 올라가기, 걷기)로 작업하고 있다는 것을 기억하십시오.

In [ ]:
# 1D CNN neural network
model_m = Sequential()
model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
model_m.add(Conv1D(100, 10, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Dropout(0.3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())

## Fit the DNN Model in Keras

다음으로, 이전에 준비한 훈련 데이터를 사용하여 모델을 맞추고/train 시킬 것입니다. 여기서는 train 정확도를 모니터링하는 사전 정지 콜백 모니터를 정의할 것입니다. 즉, 훈련이 2개의 연속적인 epochs 동안 개선되지 않으면 best model을 가지고 훈련을 중지합니다. 훈련에 사용되는 하이퍼파라미터는 매우 간단합니다. 400개의 레코드로 된 batch size를 사용하고 50 epochs 동안에 모델을 훈련시킬 것입니다. 모델 훈련을 위해 80:20 분할을 사용하여 훈련 데이터와 검증 데이터를 분리합니다. 정말 간단합니다. 그러면 모델을 훈련시켜 보겠습니다. 서로 다른 hyperparameters에 대해서 설명은 아래 링크를 참고하세요. [예제](https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9)

In [ ]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model_CNN/best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

# Hyper-parameters
BATCH_SIZE = 400
EPOCHS = 50

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
history = model_m.fit(x_train,
                      y_train_hot,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['acc'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_acc'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# training data에 대한 confusion matrix를 출력
y_pred_train = model_m.predict(x_train)
# train 예측에서 가장 놓은 확률을 가진 class를 가져온다.
max_y_pred_train = np.argmax(y_pred_train, axis=1)
print(classification_report(y_train, max_y_pred_train))


## Test 데이터 확인하기

이 모델을 계속 진행하고 이전에 따로 보관해둔 테스트 데이터에 대한 성능을 확인해보겠습니다. 저희의 경우에는 모델이 아직 보지 못한 여섯 사용자의 동작에 대한 성능을 확인할 것입니다.
Let's continue with this model and see how it performs against the test data that we have held back earlier. In our case, we will check the performance against the movements of the six users that the model has not yet seen.

In [ ]:
# Normalize features for training data set
df_test['x-axis'] = df_test['x-axis'] / df_test['x-axis'].max()
df_test['y-axis'] = df_test['y-axis'] / df_test['y-axis'].max()
df_test['z-axis'] = df_test['z-axis'] / df_test['z-axis'].max()

df_test = df_test.round({'x-axis': 4, 'y-axis': 4, 'z-axis': 4})

x_test, y_test = create_segments_and_labels(df_test,
                                            TIME_PERIODS,
                                            STEP_DISTANCE,
                                            LABEL)

# Set input_shape / reshape for Keras
x_test = x_test.reshape(x_test.shape[0], input_shape)

x_test = x_test.astype('float32')
y_test = y_test.astype('float32')

y_test = np_utils.to_categorical(y_test, num_classes)

score = model_m.evaluate(x_test, y_test, verbose=1)

print('\nAccuracy on test data: %0.2f' % score[1])
print('\nLoss on test data: %0.2f' % score[0])

테스트 데이터에 대한 정확도는 76%입니다. 이는 모델이 새로운 개체들에게도 잘 일반화된다는 것을 의미합니다. 모델이 잘못 예측한 레이블을 살펴보겠습니다.

In [ ]:
def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

y_pred_test = model_m.predict(x_test)
# Take the class with the highest probability from the test predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)

print(classification_report(max_y_test, max_y_pred_test))

보시다시피, 모델의 정확도는 달라기(1), 앉기(2), 서기(3), 걷기(5)를 예측하는 데 좋습니다. 모델은 오르기과 내려가기 활동을 명확하게 식별하는 데 문제가 있습니다. 

물론 모델을 개선할 수 있는 큰 잠재력이 여전히 있습니다. 예를 들어, 컨볼루션 신경망(CNN)이나 장단기 메모리(LSTM)와 같은 더 고급 신경망을 사용하는 것입니다.

## Prediction 

CoreML 모델을 사용하기 전에, export가 성공했는지 그리고 Keras 모델과 CoreML 모델 모두 랜덤 데이터 세트가 제공될 때 동일한 예측을 제공하는지 확인합시다.

In [ ]:
test_record = x_test[1].reshape(1,input_shape)
keras_prediction = np.argmax(model_m.predict(test_record), axis=1)
print('\nPrediction:\t',le.inverse_transform(keras_prediction)[0])
print('\nTruth:\t\t',LABELS[np.argmax(y_test[1])])